## Get cumulative number of peaks discovered with increasing number of tissues
(greedy algorithm)
a. mRNA TSS  (from TSS.bed)  
b. unstable TSS  (+/- 1kb with a tags cutoff- or add confident/optimal)  
c. lncRNA - Promoters annotatePeaks  
d. miRNA  - Promoters annotatePeaks  
e. snoRNA - Promoters annotatePeaks  
f. other  - Intron, divergent, in CDS   
g. ATAC peaks  - atac_merge.bed  

In [1]:
import yaml
import os
import pandas as pd
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'

samples = pd.read_csv('../parameters/samples.tsv',sep='\t', index_col=0)
import sys
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
#anno_gff=doc['annotation']
os.chdir(doc["results"])
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']

import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *
import helper
import create_output
print('Number of tissues: ',len(tissues))

from tss.data import generate_genome
homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # Remove the CWD from sys.path while we load stuff.


Number of tissues:  13
params {'EMAIL': 'isshamie@ucsd.edu', 'parameters': 'parameters/params.yaml'}
/data2/resources/software/homer/v4.11_10242019/bin//
/data2/resources/software/homer/v4.11_10242019/bin//homer


In [2]:
alt_anno_f = doc["alt_annotation"]
print(alt_anno_f)
alt_anno_df = pd.read_csv(alt_anno_f,sep="\t", header=None,comment="#")
anno_types = alt_anno_df[2].unique()
not_include = ['mRNA','gene', 'region','exon','CDS','primary_transcript','guide_RNA',
               'pseudogene','transcript','match','cDNA_match']
anno_types_keep = set(anno_types) - set(not_include)

print(anno_types_keep)
alt_anno_df_otherrna = alt_anno_df[alt_anno_df[2].isin(anno_types_keep)]
print(alt_anno_df_otherrna.shape)
alt_anno_df_otherrna = generate_genome.expand_anno_id(alt_anno_df_otherrna)
alt_anno_df_otherrna


/data/isshamie/genome/hamster/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3


222it [00:00, 2217.45it/s]

{'snRNA', 'lnc_RNA', 'snoRNA', 'rRNA', 'C_gene_segment', 'miRNA', 'tRNA', 'V_gene_segment'}
(8767, 9)


8767it [00:02, 3767.27it/s]


,0,1,2,3,4,5,6,7,8,ID,...,product,transcript_id,Note,anticodon,inference,standard_name,exception,end_range,partial,start_range
4,NW_020822366.1,Gnomon,lnc_RNA,269294,313331,.,+,.,"ID=rna0;Parent=gene1;Dbxref=GeneID:113832197,G...",rna0,...,uncharacterized LOC113832197%2C transcript var...,XR_003479383.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NW_020822366.1,Gnomon,lnc_RNA,551299,551829,.,+,.,"ID=rna1;Parent=gene2;Dbxref=GeneID:113832198,G...",rna1,...,uncharacterized LOC113832198,XR_003479384.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NW_020822366.1,Gnomon,lnc_RNA,696459,713531,.,+,.,"ID=rna2;Parent=gene3;Dbxref=GeneID:103159926,G...",rna2,...,uncharacterized LOC103159926%2C transcript var...,XR_003479386.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NW_020822366.1,Gnomon,lnc_RNA,696459,713531,.,+,.,"ID=rna3;Parent=gene3;Dbxref=GeneID:103159926,G...",rna3,...,uncharacterized LOC103159926%2C transcript var...,XR_003479388.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NW_020822366.1,Gnomon,lnc_RNA,710127,713531,.,+,.,"ID=rna4;Parent=gene3;Dbxref=GeneID:103159926,G...",rna4,...,uncharacterized LOC103159926%2C transcript var...,XR_003479385.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277584,NW_020824166.1,Gnomon,V_gene_segment,30244,32178,.,+,.,ID=id632731;Parent=gene32998;Dbxref=GeneID:113...,id632731,...,NaN,NaN,NaN,NaN,NaN,Ig heavy chain V region 3-6-like,NaN,NaN,NaN,NaN
1277593,NW_020824167.1,tRNAscan-SE,tRNA,20889,20960,.,+,.,ID=rna56292;Parent=gene32999;Dbxref=GeneID:107...,rna56292,...,tRNA-Ala,NaN,transfer RNA alanine (anticodon UGC),(pos:20921..20923),COORDINATES: profile:tRNAscan-SE:1.23,NaN,NaN,NaN,NaN,NaN
1277684,NW_020824187.1,Gnomon,V_gene_segment,13199,14646,.,-,.,ID=id632785;Parent=gene33011;Dbxref=GeneID:113...,id632785,...,NaN,NaN,NaN,NaN,NaN,Ig kappa chain V-II region 26-10-like,NaN,NaN,NaN,NaN
1277741,NW_020824193.1,Gnomon,V_gene_segment,12876,13513,.,+,.,ID=id632813;Parent=gene33016;Dbxref=GeneID:113...,id632813,...,NaN,NaN,NaN,NaN,NaN,Ig kappa chain V-III region MOPC 63-like,NaN,NaN,NaN,NaN


## Load peak annotations and expression

In [3]:
annoPeaks_f = join('analysis_ncbi_picr', 'merged','samples.merge.anno')
peaks_expr_f = join('analysis_ncbi_picr','merged','tissues.merge.peaksexpression')
peaks_with_distance_noCDS = 'analysis_ncbi_picr/merged/peaks_with_tss_distances_size1.noCDS.tsv'

In [4]:
expression_df = pd.read_csv(peaks_expr_f,sep="\t",index_col=0)

annoPeaks = pd.read_csv(annoPeaks_f,sep="\t",index_col=0)
annoPeaks = annoPeaks.dropna(subset=['Annotation'])
#annoPeaks = annoPeaks[annoPeaks["Annotation"].str.contains("promoter")]
annoPeaks

,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,Nearest PromoterID,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type
PeakID (cmd=annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3),,,,,,,,,,,,,,,,,,
539323,NW_020822460.1,1084630,1084782,+,66162.929688,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id200258;Parent=rna17072;Dbxr...,NaN,-1.0,ID=id200258;Parent=rna17072;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353477,NW_020822618.1,401845,402000,-,21905.132812,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id559499;Parent=rna48773;Dbxr...,NaN,4.0,ID=id559499;Parent=rna48773;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,NW_020822410.1,4827637,4827815,+,20179.468750,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id67671;Parent=rna5683;Dbxref...,NaN,-11.0,ID=id67671;Parent=rna5683;Dbxref=GeneID:100756...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349134,NW_020822505.1,689618,689773,-,18150.236328,f04_peaks/Brain_GROCap1_and_Brain_GRO1|f04_pea...,promoter-TSS (ID=id298638;Parent=rna25827;Dbxr...,NaN,3.0,ID=id298639;Parent=rna25828;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300005,NW_020822602.1,9283364,9283514,+,15297.872070,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id489748;Parent=rna42896;Dbxr...,NaN,0.0,ID=id489748;Parent=rna42896;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289272,NW_020822370.1,10486847,10486997,+,0.299000,f04_peaks/CHO_START1_and_CHO_startinput1,Intergenic,NaN,-6013.0,ID=id6288;Parent=rna501;Dbxref=GeneID:10077391...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285035,NW_020822597.1,36051369,36051519,+,0.299000,f04_peaks/CHO_START1_and_CHO_startinput1,Intergenic,NaN,20476.0,ID=id460553;Parent=rna40137;Dbxref=GeneID:1079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305615,NW_020822519.1,31154352,31154647,-,0.294000,f04_peaks/CHO_START1_and_CHO_startinput1|f04_p...,Intergenic,NaN,-62774.0,ID=id337466;Parent=rna29527;Dbxref=GeneID:1007...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Get only ncRNA 

In [5]:
ncrnas = annoPeaks[annoPeaks['Annotation'].fillna('').str.contains("ncRNA")]
ncrnas_prom = ncrnas[(ncrnas["Annotation"].fillna('').str.contains("promoter"))]
ncrnas_prom_expression = expression_df.loc[ncrnas_prom.index]
print(f"Number of ncrnas: {ncrnas.shape[0]}")
ncrnas_prom_expression = expression_df.loc[ncrnas_prom.index]

for curr_anno in ncrnas["Annotation"].values:
    print(curr_anno)
    break


print(len(ncrnas_prom.index))
print(len(set(ncrnas_prom.index)))


Number of ncrnas: 13102
promoter-TSS (ID=id200258;Parent=rna17072;Dbxref=GeneID:113834114,RFAM:RF00003,Genbank:XR_003482412.1;gbkey=ncRNA;gene=LOC113834114;inference=COORDINATES: profile:INFERNAL:1.1.1;product=U1 spliceosomal RNA;transcript_id=XR_003482412.1)
7862
7862


### Nonpromoter (TTS, exon, etc)

In [6]:
ncrnas[~(ncrnas["Annotation"].fillna('').str.contains("promoter"))]


,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,Nearest PromoterID,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type
PeakID (cmd=annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3),,,,,,,,,,,,,,,,,,
21983,NW_020822505.1,2866296,2866446,+,694.626953,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,exon (ID=id300913;Parent=rna26020;Dbxref=GeneI...,NaN,171.0,ID=id300913;Parent=rna26020;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238325,NW_020822601.1,16126915,16127065,+,365.425018,f04_peaks/Brain_GROCap1_and_Brain_GRO1|f04_pea...,TTS (ID=id469462;Parent=rna41058;Dbxref=GeneID...,NaN,156.0,ID=id469462;Parent=rna41058;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307532,NW_020822505.1,2866149,2866332,-,276.271881,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,TTS (ID=id300913;Parent=rna26020;Dbxref=GeneID...,NaN,302.0,ID=id300913;Parent=rna26020;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312568,NW_020822701.1,6736061,6736332,-,266.550995,f04_peaks/Brain_GROCap1_and_Brain_GRO1|f04_pea...,TTS (ID=id622738;Parent=rna55027;Dbxref=GeneID...,NaN,325.0,ID=id622748;Parent=rna55029;Dbxref=GeneID:1031...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491169,NW_020822658.1,11794505,11794655,-,152.279999,f04_peaks/CHO_GROCap2_and_CHO_GRO1|f04_peaks/C...,TTS (ID=id600106;Parent=rna52798;Dbxref=GeneID...,NaN,1000.0,ID=id600106;Parent=rna52798;Dbxref=GeneID:1138...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430755,NW_020822439.1,25030902,25031052,+,0.339000,f04_peaks/CHO_GROCap1_and_CHO_GRO1,TTS (ID=id140944;Parent=rna12170;Dbxref=GeneID...,NaN,892.0,ID=id140944;Parent=rna12170;Dbxref=GeneID:1079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430477,NW_020822415.1,2190682,2190832,-,0.339000,f04_peaks/CHO_GROCap1_and_CHO_GRO1,TTS (ID=id73331;Parent=rna6217;Dbxref=GeneID:1...,NaN,605.0,ID=id73334;Parent=rna6218;Dbxref=GeneID:103161...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429815,NW_020822601.1,31380102,31380252,-,0.339000,f04_peaks/CHO_GROCap1_and_CHO_GRO1,TTS (ID=id472513;Parent=rna41338;Dbxref=GeneID...,NaN,684.0,ID=id472513;Parent=rna41338;Dbxref=GeneID:1031...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Need to go back to gff3 file to see what type of ncRNA the rna ID is.

In [7]:
ncrnas_prom = ncrnas_prom.dropna(axis=1,how='all')
ncrnas_prom = generate_genome.expand_anno_id(ncrnas_prom,colname='Nearest PromoterID')
ncrnas_prom

7862it [00:01, 4161.77it/s]


,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Distance to TSS,Nearest PromoterID,ID,...,gene,inference,product,transcript_id,Note,exception,anticodon,model_evidence,end_range,partial
PeakID (cmd=annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3),,,,,,,,,,,,,,,,,,,,,
539323,NW_020822460.1,1084630,1084782,+,66162.929688,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id200258;Parent=rna17072;Dbxr...,-1.0,ID=id200258;Parent=rna17072;Dbxref=GeneID:1138...,id200258,...,LOC113834114,COORDINATES: profile:INFERNAL:1.1.1,U1 spliceosomal RNA,XR_003482412.1,NaN,NaN,NaN,NaN,NaN,NaN
353477,NW_020822618.1,401845,402000,-,21905.132812,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id559499;Parent=rna48773;Dbxr...,4.0,ID=id559499;Parent=rna48773;Dbxref=GeneID:1138...,id559499,...,LOC113836945,COORDINATES: profile:INFERNAL:1.1.1,U7 small nuclear RNA,XR_003487470.1,NaN,NaN,NaN,NaN,NaN,NaN
349134,NW_020822505.1,689618,689773,-,18150.236328,f04_peaks/Brain_GROCap1_and_Brain_GRO1|f04_pea...,promoter-TSS (ID=id298638;Parent=rna25827;Dbxr...,3.0,ID=id298639;Parent=rna25828;Dbxref=GeneID:1138...,id298639,...,LOC113831472,COORDINATES: profile:INFERNAL:1.1.1,small nucleolar RNA SNORD29,XR_003484049.1,NaN,NaN,NaN,NaN,NaN,NaN
300005,NW_020822602.1,9283364,9283514,+,15297.872070,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id489748;Parent=rna42896;Dbxr...,0.0,ID=id489748;Parent=rna42896;Dbxref=GeneID:1138...,id489748,...,LOC113830843,COORDINATES: profile:INFERNAL:1.1.1,small nucleolar RNA SNORD36,XR_003486731.1,NaN,NaN,NaN,NaN,NaN,NaN
509442,NW_020822421.1,11363589,11363741,+,11703.328125,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,promoter-TSS (ID=id96305;Parent=rna8174;Dbxref...,-2.0,ID=id96305;Parent=rna8174;Dbxref=GeneID:113833...,id96305,...,LOC113833169,COORDINATES: profile:INFERNAL:1.1.1,U11 spliceosomal RNA,XR_003480724.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429335,NW_020822529.1,14608992,14609142,-,0.339000,f04_peaks/CHO_GROCap1_and_CHO_GRO1,promoter-TSS (ID=id349527;Parent=rna30742;Dbxr...,30.0,ID=id349527;Parent=rna30742;Dbxref=GeneID:1079...,id349527,...,LOC107978526,NaN,uncharacterized LOC107978526%2C transcript var...,XR_003484941.1,NaN,NaN,NaN,NaN,NaN,NaN
430896,NW_020822555.1,2913588,2913738,-,0.339000,f04_peaks/CHO_GROCap1_and_CHO_GRO1,promoter-TSS (ID=id406076;Parent=rna35491;Dbxr...,742.0,ID=id406155;Parent=rna35497;Dbxref=GeneID:1007...,id406155,...,Plxna2,NaN,plexin A2%2C transcript variant X1,XM_027416565.1,NaN,NaN,NaN,NaN,NaN,NaN
343309,NW_020822698.1,6390597,6390747,-,0.325000,f04_peaks/CHO_GROCap2_and_CHO_GRO1,promoter-TSS (ID=id618829;Parent=rna54640;Dbxr...,-494.0,ID=id618829;Parent=rna54640;Dbxref=GeneID:1138...,id618829,...,LOC113837741,NaN,uncharacterized LOC113837741,XR_003488458.1,NaN,NaN,NaN,NaN,NaN,NaN


## ID to Parent seems to work
Get the peak IDs for each biotype

In [8]:
gene_ID = dict()
for biotype, df in alt_anno_df_otherrna.groupby(2):
    print(biotype)
    gene_ID[biotype] = df['ID'].values

peak_ID = dict()
for i in gene_ID:
    print(i)
    print(f"Number of the type: {len(gene_ID[i])}")
#     print('gene')
#     print(ncrnas_prom['gene'].isin(gene_ID[i]).sum())

#     print('transcript_id')
#     print(ncrnas_prom['transcript_id'].isin(gene_ID[i]).sum())
#     print('ID')
#     print(ncrnas_prom['ID'].isin(gene_ID[i]).sum())
    print('Parent')
    print(ncrnas_prom['Parent'].isin(gene_ID[i]).sum())

    peak_ID[i] = ncrnas_prom[ncrnas_prom['Parent'].isin(gene_ID[i])].index

peak_ID

C_gene_segment
V_gene_segment
lnc_RNA
miRNA
rRNA
snRNA
snoRNA
tRNA
C_gene_segment
Number of the type: 9
Parent
0
V_gene_segment
Number of the type: 592
Parent
0
lnc_RNA
Number of the type: 5891
Parent
5959
miRNA
Number of the type: 291
Parent
53
rRNA
Number of the type: 22
Parent
0
snRNA
Number of the type: 871
Parent
466
snoRNA
Number of the type: 606
Parent
219
tRNA
Number of the type: 485
Parent
20


{'C_gene_segment': Int64Index([], dtype='int64', name='PeakID (cmd=annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3)'),
 'V_gene_segment': Int64Index([], dtype='int64', name='PeakID (cmd=annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3)'),
 'lnc_RNA': Int64Index([484565, 129410, 195062,  88219, 450911, 113553,   6896, 346530,
             146151, 276873,
             ...
             428483, 428439, 428957, 429708, 428485, 429921, 429335, 343309,
             344967, 429753],
            dtype='int64', name='PeakID (cmd=annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_l

## Input:
a. Peak-tissue expression matrix
b. Peak annotation matrix to get the ID of RNA
c. genome annotation to get the type of RNA

## Get the number of peaks with increasing n


In [ ]:
biotype_cum_peaks = dict()
for i in peak_ID:
    curr_bin_df = create_binary_from_expr(expr_df.loc[peak_ID[i]],threshold=1)

    # Tissue peaks
    curr_tissue_peaks = {}
    for t in gene_tissues_order:    
        col = t
        curr_tissue_peaks[t] = set(curr_bin_df[~(curr_bin_df[col]==0)].index.values)

    
    curr_cum_peaks_n, curr_cum_peaks = get_cum_and_plot(curr_tissue_peaks, gene_tissues_order);
    biotype_cum_peaks_n[i] = curr_cum_peaks_n

In [ ]:
bin_df_enh = create_binary_from_expr(expr_df,threshold=1)
#cdf_number,cdf_tissues = create_cdf(bin_df_enh)

distal_peaks = {}
for t in gene_tissues_order:    
    col = t
    distal_peaks[t] = set(bin_df_enh[~(bin_df_enh[col]==0)].index.values)

enh_cum_peaks_n, enh_cum_peaks = get_cum_and_plot(distal_peaks, gene_tissues_order);
